In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to buy']
def stock_list(values):
    df = pd.DataFrame(columns=my_columns)
        
    for name in stocks['Ticker'][:1]:
        api_url = f'https://sandbox.iexapis.com/stable/stock/{name}/quote?token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(api_url).json()
    
        df = df.append(
            pd.Series(
                [
                    name,
                    data['latestPrice'],
                    data['marketCap'],
                    'N/A'
                    ],
                    index=my_columns
                    ),
            ignore_index=True
            )
    # df = pd.concat(holder)
    # print(data)
    return df
data = stock_list(stocks)
data

,Ticker,Stock Price,Market Capitalization,Number of Shares to buy
0,A,146.61,44851375964,N/A


In [4]:
def chunks(lst, n):
    
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [5]:
def batch_call(value):
    symbols = list(chunks(stocks['Ticker'], 100))
    symbol_strings = []
    
    for i in range(0, len(symbols)):
        symbol_strings.append(','.join(symbols[i]))
    
    df = pd.DataFrame(columns=my_columns)
    for name in symbol_strings:
        
        batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={name}&types=quote&token={IEX_CLOUD_API_TOKEN}'
        datas = requests.get(batch_api_call_url).json()
        
        for quote in name.split(','):
            try:
                df = df.append(
                pd.Series(
                    [
                        quote,
                        datas[quote]['quote']['latestPrice'],
                        datas[quote]['quote']['marketCap'],
                        'N/A'   
                    ],
                index=my_columns),
                ignore_index = True
                )
            except:
                # print(quote) 
                continue
    return df

dataset = batch_call(stocks)
dataset

,Ticker,Stock Price,Market Capitalization,Number of Shares to buy
0,A,147.41,44553870312,N/A
1,AAL,13.89,9285927312,N/A
2,AAP,148.36,9049581347,N/A
3,AAPL,148.39,2416614872215,N/A
4,ABBV,162.7,282520870618,N/A
...,...,...,...,...
496,YUM,126.46,35260522930,N/A
497,ZBH,116.89,24575657003,N/A
498,ZBRA,265,13647294936,N/A
499,ZION,52.05,7816936534,N/A


In [6]:
# portfolio_size = input('portfolio size??')
portfolio_size = 1000000

try:
    val = float(portfolio_size)
except:
    print("That's not a number \nPlease Try again")
    portfolio_size = input('portfolio size??')
    val = float(portfolio_size)


position_size = val/len(dataset.index)

for i in range(len(dataset.index)):
    dataset.loc[i, "Number of Shares to buy"] = math.floor(position_size/dataset.loc[i, 'Stock Price'])
    
dataset

,Ticker,Stock Price,Market Capitalization,Number of Shares to buy
0,A,147.41,44553870312,13
1,AAL,13.89,9285927312,143
2,AAP,148.36,9049581347,13
3,AAPL,148.39,2416614872215,13
4,ABBV,162.7,282520870618,12
...,...,...,...,...
496,YUM,126.46,35260522930,15
497,ZBH,116.89,24575657003,17
498,ZBRA,265,13647294936,7
499,ZION,52.05,7816936534,38


In [7]:
dataset.to_excel('recommended_trades.xlsx')